# Olympic Dataset

* **Installing pyspark**

In [1]:
!pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 54.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ea469b9ad80195f28c3b0d45caa44e3a28a9124dc29eedfba52876db3e221711
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


* **Creating Environmemt for pyspark**

In [2]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
conf=SparkConf().setAppName('test1')
sc=SparkContext(conf=conf)
spark=SparkSession.builder.appName('test2').getOrCreate()
sqlContext=SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


* **Importing libraries**

In [3]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re
import random
import pyspark.sql.functions as f
from pyspark.sql.functions import udf,col
from pyspark.sql.types import IntegerType

* **Write PySpark code to read olympix_data.csv file**


In [30]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load('/content/olympic_dataset_final.csv')
df.show(10) 
"""Here I am reading the file using spark.The .show() function is used to display the dataframe and passing arguments inside the parenthesis will display that
   many rows.For eg: df.show(10) displays the first 10 rows"""

+---+----------------+----+-------------+----+----------+----------+----------+------------+-----------+------------+
|_c0|            name| age|      country|year|Date_Given|    sports|gold_medal|silver_medal|total_medal|bronze_medal|
+---+----------------+----+-------------+----+----------+----------+----------+------------+-----------+------------+
|  0|  Michael Phelps|23.0|United States|2008| 8/24/2008|  Swimming|         8|           0|          8|           0|
|  1|  Michael Phelps|19.0|United States|2004| 8/29/2004|  Swimming|         6|           0|          8|           2|
|  2|  Michael Phelps|27.0|United States|2012|08-12-2012|  Swimming|         4|           2|          6|           0|
|  3|Natalie Coughlin|25.0|United States|2008| 8/24/2008|  Swimming|         1|           2|          6|           3|
|  4|   Aleksey Nemov|24.0|       Russia|2000|10-01-2000|Gymnastics|         2|           1|          6|           3|
|  5|   Alicia Coutts|24.0|    Australia|2012|08-12-2012

In [31]:
df.printSchema() #.printSchema displays the datatypes of all the columns

root
 |-- _c0: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: double (nullable = true)
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- Date_Given: string (nullable = true)
 |-- sports: string (nullable = true)
 |-- gold_medal: integer (nullable = true)
 |-- silver_medal: integer (nullable = true)
 |-- total_medal: integer (nullable = true)
 |-- bronze_medal: integer (nullable = true)



* **Write PySpark code to print the Olympic Sports/games in the dataset.**

In [ ]:
df.select(df.sports).show(100) #Here I have displayed the values of the 'sports' column


* **Write PySpark code to plot the total number of medals in each Olympic Sport/game**

In [34]:
#Visualizations cannot be performed in pyspark so first I have converted the Pyspark DataFrame to Pandas DataFrame
new_df=df.toPandas()

In [ ]:
plt.figure(figsize=(20,30))#plt.figure is used to set the width & height of the plot
plt.title("Total number of medals in each Olympic Sport")#Used to give Title to your plot
sns.violinplot(x='total_medal',y='sports',data=new_df,color='b')#Here I have used the violinplot to display the total no. of medals in each Olympic Sport
plt.show()#Used to display the plot

* **Sort the result based on the total number of medals.**

In [36]:
dfj = df.groupBy('sports').sum('total_medal') 
dfj.orderBy(f.desc('sum(total_medal)')).show()
#Here I have grouped by total number of medals with respect to sports and then sorted the result of this in descending order of total number of medals 

+----------+----------------+
|    sports|sum(total_medal)|
+----------+----------------+
|  Swimming|             765|
| Athletics|             753|
|    Rowing|             576|
|  Football|             407|
|    Hockey|             388|
|Ice Hockey|             384|
|  Handball|             351|
|  Canoeing|             333|
| Waterpolo|             306|
|   Cycling|             306|
|Basketball|             287|
|Gymnastics|             282|
|Volleyball|             281|
|   Fencing|             256|
| Wrestling|             245|
|      Judo|             224|
|  Baseball|             216|
|   Sailing|             210|
|  Shooting|             192|
|    Boxing|             188|
+----------+----------------+
only showing top 20 rows



* **Find the total number of medals won by each country in swimming.**

In [ ]:
df_new=df.where(df.sports=='Swimming').select('sports','country','total_medal')
df_new.show()
"""To solve this first I have filtered sports column with 'Swimming' value and then displayed 'sports','country','total_medal' for this filtered data and 
then grouped by the result with 'country','sports' for total number of medalswhich gives total no. of medals in swimming per country """

In [ ]:
dfq=df_new.groupBy('country','sports').sum('total_medal')
dfq.orderBy(f.desc('sum(total_medal)')).show()

* **Find the total number of medals won by each country in Skeleton.**

In [ ]:
df_new_1=df.where(df.sports=='Skeleton').select('sports','country','total_medal')
df_new_1.show() 
"""To solve this first I have filtered sports column with 'Skeleton' value and then displayed 'sports','country','total_medal' for this filtered data and       
  then grouped by the result with 'country','sports' for total number of medals which gives total no. of medals in skeleton per country"""

In [40]:
dfh=df_new_1.groupBy('country','sports').sum('total_medal')
dfh.orderBy(f.desc('sum(total_medal)')).show()

+-------------+--------+----------------+
|      country|  sports|sum(total_medal)|
+-------------+--------+----------------+
|       Canada|Skeleton|               4|
|United States|Skeleton|               3|
|  Switzerland|Skeleton|               3|
|Great Britain|Skeleton|               3|
|      Germany|Skeleton|               2|
|       Russia|Skeleton|               1|
|      Austria|Skeleton|               1|
|       Latvia|Skeleton|               1|
+-------------+--------+----------------+



* **Find the number of medals that the US won yearly.**

In [41]:
df_new_2=df.where(df.country=='United States').select('year','country','gold_medal','silver_medal','bronze_medal','total_medal')
df_new_2.show()
"""To solve this first I have filtered country column with 'United States' value and then displayed 'year','country','total_medal' for this filtered data and 
then grouped by the result with 'country','year' for total number of medalsand finally ordered the result of this in ascending order of year which gives total
 no. of medals that the US won yearly."""

+----+-------------+----------+------------+------------+-----------+
|year|      country|gold_medal|silver_medal|bronze_medal|total_medal|
+----+-------------+----------+------------+------------+-----------+
|2008|United States|         8|           0|           0|          8|
|2004|United States|         6|           0|           2|          8|
|2012|United States|         4|           2|           0|          6|
|2008|United States|         1|           2|           3|          6|
|2012|United States|         4|           0|           1|          5|
|2012|United States|         2|           2|           1|          5|
|2012|United States|         3|           1|           1|          5|
|2004|United States|         2|           2|           1|          5|
|2000|United States|         2|           0|           3|          5|
|2008|United States|         1|           3|           1|          5|
|2008|United States|         2|           0|           2|          4|
|2000|United States|

In [45]:
dfx=df_new_2.groupBy('country','year').sum('gold_medal','silver_medal','bronze_medal','total_medal')
dfx.orderBy(f.asc('year')).show()

+-------------+----+---------------+-----------------+-----------------+----------------+
|      country|year|sum(gold_medal)|sum(silver_medal)|sum(bronze_medal)|sum(total_medal)|
+-------------+----+---------------+-----------------+-----------------+----------------+
|United States|2000|            130|               61|               52|             243|
|United States|2002|             11|               58|               15|              84|
|United States|2004|            118|               75|               72|             265|
|United States|2006|              9|               11|               32|              52|
|United States|2008|            127|              109|               81|             317|
|United States|2010|             12|               63|               22|              97|
|United States|2012|            145|               63|               46|             254|
+-------------+----+---------------+-----------------+-----------------+----------------+



* **Find the total number of medals won by each country.**

In [46]:
dfs=df.groupBy('country').sum('total_medal')
dfs.orderBy(f.desc('sum(total_medal)')).show()
#To solve this I have grouped by total no.of medals with country and then ordered the result of this in descending order of total no. of medals

+-------------+----------------+
|      country|sum(total_medal)|
+-------------+----------------+
|United States|            1312|
|       Russia|             768|
|      Germany|             629|
|    Australia|             609|
|        China|             530|
|       Canada|             370|
|        Italy|             331|
|Great Britain|             322|
|       France|             318|
|  Netherlands|             318|
|  South Korea|             308|
|        Japan|             282|
|       Brazil|             221|
|        Spain|             205|
|       Norway|             192|
|         Cuba|             188|
|       Sweden|             181|
|      Hungary|             145|
|      Ukraine|             143|
|    Argentina|             141|
+-------------+----------------+
only showing top 20 rows



* **Who was the oldest athlete in the olympics and which country was he/she from?**

In [ ]:
df.agg({'age':'max'}).show()
"""To solve this first I calculated maximum value of age & then filtered the data for that value and displayed name,age and country for that value which gives
 me the name of oldest athlete in olympics"""

+--------+
|max(age)|
+--------+
|    61.0|
+--------+



In [47]:
ddf=df.filter(df.age==61) 
df_old=ddf.select('name','age','country')
df_old.show()

+----------+----+-------+
|      name| age|country|
+----------+----+-------+
|Ian Millar|61.0| Canada|
+----------+----+-------+



In [ ]:
ddf1=df.where(df.age==61).select('name','age','country')
ddf1.show() #alternative code for the above question

+----------+----+-------+
|      name| age|country|
+----------+----+-------+
|Ian Millar|61.0| Canada|
+----------+----+-------+

